In [ ]:

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/areaverde.csv')
df['Localidade'] = df['Localidade'].str.upper()
df = df.set_index('Localidade')

df2 = pd.read_csv('data/populacao.csv', index_col=False)
df2['Localidade'] = df2['Localidade'].str.upper()

df3 = pd.read_csv('data/renda.csv', index_col=False)
df3['Localidade'] = df3['Localidade'].str.upper()

df4 = pd.read_csv('data/covid.csv', index_col=False)
df5 = pd.read_csv('data/dengue.csv', index_col=False)
df6 = pd.read_csv('data/num_atendimentos.csv', index_col=False)
df7 = pd.read_csv('data/num_unidades_saude.csv', index_col=False)
df7['Localidade'] = df7['Localidade'].str.upper()


df = df.merge(df2, on='Localidade', how='left')
df = df.merge(df3, on='Localidade', how='left')

df = df.merge(df4, on='Localidade', how='left')
df = df.merge(df5, on='Localidade', how='left')
df = df.merge(df6, on='Localidade', how='left')
df = df.merge(df7, on='Localidade', how='left', suffixes=('_x1', '_x2'))

df['Localidade'] = df['Localidade'].str.upper()

# df['porcentagem_cobertura'] = df['area_verde'] / df['area']

df['porcentagem_brancos'] = df['popBrancos'] / df['popTotal']
df['porcentagem_adultos'] = df['pop15a64'] / df['popTotal']

df['taxa_obitos_covid'] = df['num_obitos_covid'] / df['popTotal']
df['taxa_atendimentos_covid'] = df['num_atendimentos_covid'] / df['popTotal']
df['taxa_atendimentos_dengue'] = df['num_atendimentos_dengue'] / df['popTotal']
df['taxa_atendimentos_saude'] = df['num_atendimentos_saude_ja'] / df['popTotal']
df['taxa_unidades_saude'] = df['num_unidades_saude_bairro'] / df['popTotal']

df = df.fillna(0)
df


In [ ]:
import ipywidgets as widgets
from IPython.display import display


gdf = gpd.read_file('DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp')
gdf = gdf.merge(df, left_on='NOME', right_on='Localidade', how='right')
gdf['area'] = gdf.area
gdf['area_verde'] = gdf['areasVerdesM2'].str.replace(',', '.').astype(float)

# Convert numeric columns from string to float, handling comma decimal separator
numeric_columns = gdf.select_dtypes(include=['object']).columns
for col in numeric_columns:
    try:
        # Try converting comma decimal separator to period and convert to float
        gdf[col] = gdf[col].str.replace(',', '.').astype(float)
    except (AttributeError, ValueError):
        # Skip if column can't be converted to float
        continue


w = widgets.Dropdown(
    options=list(gdf),
    value='Localidade',
    description='Data:',
)

map_output = widgets.Output()

display(w, map_output)

In [4]:
from IPython.display import clear_output

def update_map(change):
    with map_output:
        clear_output(wait=True)
        display(gdf.explore(column=change.new, legend=True, legend_kwds={"label": change.new, "orientation": "vertical"}))

w.observe(update_map, names='value')

# Initial map display
with map_output:
    display(gdf.explore(column=w.value, legend=True, legend_kwds={"label": w.value, "orientation": "vertical"}))

In [5]:
# Save the GeoDataFrame to a GeoJSON file
# gdf.to_file("curitiba_bairros_completo.geojson", driver='GeoJSON')


In [9]:
# Create maps for each variable and save to file
for column in gdf.columns:
    if column != 'geometry':  # Skip the geometry column
        fig, ax = plt.subplots(figsize=(12, 8))
        gdf.plot(column=column, ax=ax, legend=True, cmap='viridis', edgecolor='grey', linewidth=0.2)
        plt.title(f'Mapa de {column}')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f'maps/map_{column}.png', dpi=300, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free up memory

/tmp/ipykernel_353705/2523702804.py:8: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
/tmp/ipykernel_353705/2523702804.py:8: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()
